In [30]:
from tqdm import tqdm
import pandas as pd
import spacy

In [31]:
df = pd.read_csv("../data/opinion.csv")

In [10]:
df.Sentence.iloc[0]

'Downloading books is easy, the screen is NOT too dark, the font size is adjustable and the Kindle customer service support is terrific.'

In [4]:
nlp = spacy.load('en_core_web_lg')

In [15]:
sentences = list(df.Sentence.values)
# sentences = [sent for sent in parsed_opinions.sents]

In [32]:
nlp_sentences = []
for sentence in tqdm(sentences):
    nlp_sentences.append(nlp(sentence))

100%|████████████████████████████████████████████████████████████████████████████| 14834/14834 [08:34<00:00, 28.81it/s]


In [79]:
import numpy as np

In [138]:
test = nlp_sentences[0]
for i in test:
    break

In [160]:
i.pos_

'VERB'

In [161]:
nlp.tagger.labels

('$',
 "''",
 ',',
 '-LRB-',
 '-RRB-',
 '.',
 ':',
 'ADD',
 'AFX',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'HYPH',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NFP',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 'XX',
 '_SP',
 '``')

In [163]:
nlp.tagger.labels

('$',
 "''",
 ',',
 '-LRB-',
 '-RRB-',
 '.',
 ':',
 'ADD',
 'AFX',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'HYPH',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NFP',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 'XX',
 '_SP',
 '``')

In [145]:
# tags = list(nlp.tagger.labels)
# tags_dict = {tag: i for i, tag in enumerate(tags)}

# for index, sent in tqdm(enumerate(nlp_sentences)):
#     vals = [0 for _ in range(len(tags))]
#     for token in sent:
#         vals[tags_dict[token.tag_]] += 1
#     features[index] = vals
    

pars = list(nlp.parser.labels) + list(nlp.tagger.labels)
pars_dict = {par: i for i, par in enumerate(pars)}
features = np.zeros((len(nlp_sentences), len(pars)))
errors = 0
for index, sent in tqdm(enumerate(nlp_sentences)):
    vals = [0 for _ in range(len(pars))]
    for token in sent:
        vals[pars_dict[token.tag_]] += 1
        try:
            vals[pars_dict[token.dep_]] += 1
        except:
            errors += 1
            continue
    features[index] = vals 

14834it [00:00, 24578.63it/s]


In [147]:
y = np.array(df.Opinion.values)

In [148]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate 
from sklearn.ensemble import RandomForestClassifier
import operator
import json
from nltk.tokenize import sent_tokenize


In [149]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.33, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=500, oob_score=True, n_jobs=-1,random_state=50, max_features=None)

In [150]:
model = rf_classifier.fit(X_train, y_train)
predictions = model.predict(X_test)

In [151]:
list_of_labels = sorted(list(set(y_train)))
precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
print("=================== Results ===================")
print("            Fact     Opinion                   ")
print("F1       " + str(f1))
print("Precision" + str(precision))
print("Recall   " + str(recall))
print("Accuracy " + str(accuracy))
print("===============================================")

=================== Results ===================
            Fact     Opinion                   
F1       [0.8914405  0.89604158]
Precision[0.898191   0.88963875]
Recall   [0.88479072 0.90253725]
Accuracy 0.8937908496732027


In [152]:
model = rf_classifier.fit(features, y)

In [153]:
with open('../data/disney_reviews.json') as f:
    reviews = json.load(f)

docs = reviews['Frozen II']

In [154]:
docs = [sent_tokenize(doc) for doc in docs]
docs = [x for sublist in docs for x in sublist]

In [124]:
nlp_sentences_new = []
for sentence in tqdm(docs):
    nlp_sentences_new.append(nlp(sentence))

100%|██████████████████████████████████████████████████████████████████████████████| 9677/9677 [01:44<00:00, 92.42it/s]


In [155]:
# tags = list(nlp.tagger.labels)
# tags_dict = {tag: i for i, tag in enumerate(tags)}
# new_features = np.zeros((len(docs), len(tags)))

# for index, sent in tqdm(enumerate(nlp_sentences_new)):
#     vals = [0 for _ in range(len(tags))]
#     for token in sent:
#         vals[tags_dict[token.tag_]] += 1
#     new_features[index] = vals

pars = list(nlp.parser.labels) + list(nlp.tagger.labels)
pars_dict = {par: i for i, par in enumerate(pars)}
new_features = np.zeros((len(nlp_sentences_new), len(pars)))
errors = 0
for index, sent in tqdm(enumerate(nlp_sentences_new)):
    vals = [0 for _ in range(len(pars))]
    for token in sent:
        vals[pars_dict[token.tag_]] += 1
        try:
            vals[pars_dict[token.dep_]] += 1
        except:
            errors += 1
            continue
    new_features[index] = vals 

9677it [00:00, 24186.18it/s]


In [164]:
predictions = model.predict_proba(new_features)

In [165]:
predictions

array([[0.332     , 0.668     ],
       [0.12311111, 0.87688889],
       [0.426     , 0.574     ],
       ...,
       [0.87      , 0.13      ],
       [0.96      , 0.04      ],
       [0.534     , 0.466     ]])

In [173]:
count

2195

In [177]:
count = 0
for index, val in enumerate(predictions[200:500]):
    if val[0] > .8 or val[1] > .8:
        print()
        print(val[0], val[1])
        print(nlp_sentences_new[200+index])
        print()
        count += 1


0.824 0.176
When a story is about two of more different cultures, and one of the cultures is what we associate with Europa, Disney have a bad habit of portraying the latter in a negative light, and take liberties like turning medieval settings into modern day rainbow coalitions.


0.11266666666666666 0.8873333333333333
The plot was soo thin with no purpose.


0.8392627594627595 0.16073724053724056
The songs were atrocious !!


0.818 0.182
The songs are really nothing to write home about.


0.187 0.8130000000000002
It was just basic.


0.828 0.172
We both fell asleep wishing the nightmare would end so we could leave.


0.012 0.988
This.


0.942 0.058
All the negative reviews I see complain about it either being too dark or all over the place.


0.17733333333333334 0.8226666666666667
It was neither.


0.93 0.07
When you pay four times as much for concessions in a movie theater than you would pay in a store for candy, I would like to feel as if the level of entertainment was higher in th